# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 29 2022 9:01AM,253982,10,0086271617,ISDIN Corporation,Released
1,Dec 29 2022 9:01AM,253982,10,0086271616,ISDIN Corporation,Released
2,Dec 29 2022 9:01AM,253982,10,0086271618,ISDIN Corporation,Released
3,Dec 29 2022 8:59AM,253981,10,0086268330,ISDIN Corporation,Released
4,Dec 29 2022 8:59AM,253981,10,0086268331,ISDIN Corporation,Released
5,Dec 29 2022 8:59AM,253981,10,0086268332,ISDIN Corporation,Released
6,Dec 29 2022 8:59AM,253981,10,0086268333,ISDIN Corporation,Released
7,Dec 29 2022 8:59AM,253981,10,0086268335,ISDIN Corporation,Released
8,Dec 29 2022 8:59AM,253981,10,0086268334,ISDIN Corporation,Released
9,Dec 29 2022 8:59AM,253981,10,0086268345,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,253978,Released,21
42,253979,Released,1
43,253980,Released,1
44,253981,Released,13
45,253982,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253978,NaN,21.0
253979,NaN,1.0
253980,NaN,1.0
253981,NaN,13.0
253982,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253891,0.0,4.0
253894,0.0,75.0
253896,0.0,1.0
253897,0.0,26.0
253900,0.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253891,0,4
253894,0,75
253896,0,1
253897,0,26
253900,0,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253891,0,4
1,253894,0,75
2,253896,0,1
3,253897,0,26
4,253900,0,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253891,,4
1,253894,,75
2,253896,,1
3,253897,,26
4,253900,,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation
3,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation
16,Dec 29 2022 8:53AM,253978,12,Hush Hush
37,Dec 29 2022 8:50AM,253980,12,Hush Hush
38,Dec 29 2022 8:50AM,253979,12,Hush Hush
39,Dec 29 2022 8:30AM,253953,10,Emerginnova
40,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC"
41,Dec 29 2022 8:30AM,253967,10,ISDIN Corporation
43,Dec 29 2022 8:30AM,253965,10,ISDIN Corporation
55,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation,,3
1,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation,,13
2,Dec 29 2022 8:53AM,253978,12,Hush Hush,,21
3,Dec 29 2022 8:50AM,253980,12,Hush Hush,,1
4,Dec 29 2022 8:50AM,253979,12,Hush Hush,,1
5,Dec 29 2022 8:30AM,253953,10,Emerginnova,,1
6,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC",,1
7,Dec 29 2022 8:30AM,253967,10,ISDIN Corporation,,6
8,Dec 29 2022 8:30AM,253965,10,ISDIN Corporation,,14
9,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation,3,
1,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation,13,
2,Dec 29 2022 8:53AM,253978,12,Hush Hush,21,
3,Dec 29 2022 8:50AM,253980,12,Hush Hush,1,
4,Dec 29 2022 8:50AM,253979,12,Hush Hush,1,
5,Dec 29 2022 8:30AM,253953,10,Emerginnova,1,
6,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC",1,
7,Dec 29 2022 8:30AM,253967,10,ISDIN Corporation,6,
8,Dec 29 2022 8:30AM,253965,10,ISDIN Corporation,14,
9,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation,3,
1,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation,13,
2,Dec 29 2022 8:53AM,253978,12,Hush Hush,21,
3,Dec 29 2022 8:50AM,253980,12,Hush Hush,1,
4,Dec 29 2022 8:50AM,253979,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation,3.0,NaN
1,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation,13.0,NaN
2,Dec 29 2022 8:53AM,253978,12,Hush Hush,21.0,NaN
3,Dec 29 2022 8:50AM,253980,12,Hush Hush,1.0,NaN
4,Dec 29 2022 8:50AM,253979,12,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 9:01AM,253982,10,ISDIN Corporation,3.0,0.0
1,Dec 29 2022 8:59AM,253981,10,ISDIN Corporation,13.0,0.0
2,Dec 29 2022 8:53AM,253978,12,Hush Hush,21.0,0.0
3,Dec 29 2022 8:50AM,253980,12,Hush Hush,1.0,0.0
4,Dec 29 2022 8:50AM,253979,12,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,7871834,269.0,0.0
12,761937,23.0,0.0
17,761882,3.0,1.0
19,1523748,6.0,1.0
20,253974,3.0,0.0
21,253916,1.0,0.0
22,507943,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,7871834,269.0,0.0
1,12,761937,23.0,0.0
2,17,761882,3.0,1.0
3,19,1523748,6.0,1.0
4,20,253974,3.0,0.0
5,21,253916,1.0,0.0
6,22,507943,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,269.0,0.0
1,12,23.0,0.0
2,17,3.0,1.0
3,19,6.0,1.0
4,20,3.0,0.0
5,21,1.0,0.0
6,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,269.0
1,12,Released,23.0
2,17,Released,3.0
3,19,Released,6.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,17,19,20,21,22
Status,,,,,,,
Executing,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Released,269.0,23.0,3.0,6.0,3.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,17,19,20,21,22
0,Executing,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,Released,269.0,23.0,3.0,6.0,3.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,17,19,20,21,22
0,Executing,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,Released,269.0,23.0,3.0,6.0,3.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()